In [2]:
import pandas as pd
from sqlalchemy import create_engine

#  Data Extract

In [7]:

housing_2016='/Users/maheshdivakarannair/Documents/GitHub/ETL_Project_HousingData/Resources/2016 Housing Data.xlsx'

In [8]:
housing_2016_df = pd.read_excel(housing_2016)


In [9]:
housing_2016_df.head()

,Year,AssignedID,Bank,FIPSStateCode,FIPState2,FIPSCountyCode,FIPCnty2,FIPState&County,Zip Code,City,...,FedFinStbltyPlan,AcqTyp,GSEREO,Unnamed: 87,Unnamed: 88,STCOUNTYFP,CLASSFP,ZIP,COUNTYNAME,STATE
0,2016,2016972,Indianapolis,1,1,3,3,1003,36507.0,Baldwin,...,2,1,2,NaN,NaN,1001.0,H1,36003.0,Autauga County,AL
1,2016,2004771,Atlanta,1,1,31,31,1031,36311.0,Coffee,...,2,1,2,NaN,NaN,1001.0,H1,36006.0,Autauga County,AL
2,2016,2054365,Topeka,1,1,39,39,1039,36028.0,Covington,...,2,1,2,NaN,NaN,1001.0,H1,36067.0,Autauga County,AL
3,2016,2055438,Atlanta,1,1,45,45,1045,36322.0,Dale,...,2,1,2,NaN,NaN,1001.0,H1,36066.0,Autauga County,AL
4,2016,2038851,Cincinnati,1,1,45,45,1045,36322.0,Dale,...,2,4,2,NaN,NaN,1001.0,H1,36703.0,Autauga County,AL


# Data Transform

In [10]:

final_housing_2016_df=housing_2016_df[[
    'Year','Bank','FIPSStateCode','FIPSCountyCode','Tractrat','Income','CurAreY','IncRat','LTV','MortDate','AcquDate',
    'Purpose','Product','Term','AmorTerm','First','BoAge','CoAge','Occup','Bed1','Bed2','Bed3','Bed4','Rate','Amount',
    'Front','Back','BoCreditScore','CoBoCreditScore','PMI','PropType'
]].copy()

In [11]:
final_housing_2016_df.head()

,Year,Bank,FIPSStateCode,FIPSCountyCode,Tractrat,Income,CurAreY,IncRat,LTV,MortDate,...,Bed3,Bed4,Rate,Amount,Front,Back,BoCreditScore,CoBoCreditScore,PMI,PropType
0,2016,Indianapolis,1,3,1.2000,84588,61100,1.3844,0.95,2016,...,98,98,0.0363,175750,0.1600,0.2600,4,5,0.30,PT07
1,2016,Atlanta,1,31,1.2759,124944,47600,2.6249,0.99,2015,...,98,98,0.0263,167300,0.1203,0.5000,3,3,0.00,PT01
2,2016,Topeka,1,39,1.3370,66454,47600,1.3961,0.80,2016,...,98,98,0.0400,128000,0.1346,0.1815,4,9,0.00,PT01
3,2016,Atlanta,1,45,1.1149,80424,47600,1.6896,0.97,2016,...,98,98,0.0338,377000,0.3023,0.4617,4,5,0.00,PT01
4,2016,Cincinnati,1,45,0.9984,73000,46300,1.5767,0.90,2016,...,98,98,0.0338,193500,0.1900,0.1900,5,9,0.25,PT01


In [14]:
final_housing_2016_df['stcountyfp']=final_housing_2016_df['FIPSCountyCode']+final_housing_2016_df['FIPSStateCode']

In [15]:
final_housing_2016_df['stcountyfp']=final_housing_2016_df['stcountyfp'].astype(int)

In [16]:
final_housing_2016_df.columns = map(str.lower, final_housing_2016_df.columns)

In [17]:
final_housing_2016_df.head()

,year,bank,fipsstatecode,fipscountycode,tractrat,income,curarey,incrat,ltv,mortdate,...,bed4,rate,amount,front,back,bocreditscore,cobocreditscore,pmi,proptype,stcountyfp
0,2016,Indianapolis,1,3,1.2000,84588,61100,1.3844,0.95,2016,...,98,0.0363,175750,0.1600,0.2600,4,5,0.30,PT07,4
1,2016,Atlanta,1,31,1.2759,124944,47600,2.6249,0.99,2015,...,98,0.0263,167300,0.1203,0.5000,3,3,0.00,PT01,32
2,2016,Topeka,1,39,1.3370,66454,47600,1.3961,0.80,2016,...,98,0.0400,128000,0.1346,0.1815,4,9,0.00,PT01,40
3,2016,Atlanta,1,45,1.1149,80424,47600,1.6896,0.97,2016,...,98,0.0338,377000,0.3023,0.4617,4,5,0.00,PT01,46
4,2016,Cincinnati,1,45,0.9984,73000,46300,1.5767,0.90,2016,...,98,0.0338,193500,0.1900,0.1900,5,9,0.25,PT01,46


# Data Load

In [18]:
rds_connection_string = "mahesh1:mahesh@localhost:5432/housing_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [19]:
final_housing_2016_df.to_sql(name='single_family_property_type', con=engine, if_exists='append', index=False)

In [20]:
pd.read_sql_query('select * from single_family_property_type where year=2016', con=engine).head()

,year,bank,fipsstatecode,fipscountycode,tractrat,income,curarey,incrat,ltv,mortdate,...,bed4,rate,amount,front,back,bocreditscore,cobocreditscore,pmi,proptype,stcountyfp
0,2016,Indianapolis,1,3,1.2000,84588,61100,1.3844,0.95,2016,...,98,0.0363,175750,0.1600,0.2600,4,5,0.30,PT07,4
1,2016,Atlanta,1,31,1.2759,124944,47600,2.6249,0.99,2015,...,98,0.0263,167300,0.1203,0.5000,3,3,0.00,PT01,32
2,2016,Topeka,1,39,1.3370,66454,47600,1.3961,0.80,2016,...,98,0.0400,128000,0.1346,0.1815,4,9,0.00,PT01,40
3,2016,Atlanta,1,45,1.1149,80424,47600,1.6896,0.97,2016,...,98,0.0338,377000,0.3023,0.4617,4,5,0.00,PT01,46
4,2016,Cincinnati,1,45,0.9984,73000,46300,1.5767,0.90,2016,...,98,0.0338,193500,0.1900,0.1900,5,9,0.25,PT01,46
